# Direct Marketing with Amazon SageMaker Autopilot
---

---

Kernel `Python 3 (Data Science)` works well with this notebook.

## Contents

1. [Introduction](#Introduction)
1. [Prerequisites](#Prerequisites)
1. [Downloading the dataset](#Downloading)
1. [Upload the dataset to Amazon S3](#Uploading)
1. [Setting up the SageMaker Autopilot Job](#Settingup)
1. [Launching the SageMaker Autopilot Job](#Launching)
1. [Tracking Sagemaker Autopilot Job Progress](#Tracking)
1. [Results](#Results)
1. [Cleanup](#Cleanup)

## Introduction

Amazon SageMaker Autopilot is an automated machine learning (commonly referred to as AutoML) solution for tabular datasets. You can use SageMaker Autopilot in different ways: on autopilot (hence the name) or with human guidance, without code through SageMaker Studio, or using the AWS SDKs. This notebook, as a first glimpse, will use the AWS SDKs to simply create and deploy a machine learning model.

A typical introductory task in machine learning (the "Hello World" equivalent) is one that uses a dataset to predict whether a customer will enroll for a term deposit at a bank, after one or more phone calls. For more information about the task and the dataset used, see [Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing).

Direct marketing, through mail, email, phone, etc., is a common tactic to acquire customers.  Because resources and a customer's attention are limited, the goal is to only target the subset of prospects who are likely to engage with a specific offer.  Predicting those potential customers based on readily available information like demographics, past interactions, and environmental factors is a common machine learning problem. You can imagine that this task would readily translate to marketing lead prioritization in your own organization.

This notebook demonstrates how you can use Autopilot on this dataset to get the most accurate ML pipeline through exploring a number of potential options, or "candidates". Each candidate generated by Autopilot consists of two steps. The first step performs automated feature engineering on the dataset and the second step trains and tunes an algorithm to produce a model. When you deploy this model, it follows similar steps. Feature engineering followed by inference, to decide whether the lead is worth pursuing or not. The notebook contains instructions on how to train the model as well as to deploy the model to perform batch predictions on a set of leads. Where it is possible, use the Amazon SageMaker Python SDK, a high level SDK, to simplify the way you interact with Amazon SageMaker.

Other examples demonstrate how to customize models in various ways. For instance, models deployed to devices typically have memory constraints that need to be satisfied as well as accuracy. Other use cases have real-time deployment requirements and latency constraints. For now, keep it simple.

## Prerequisites

Before you start the tasks in this tutorial, do the following:

- The Amazon Simple Storage Service (Amazon S3) bucket and prefix that you want to use for training and model data. This should be within the same Region as Amazon SageMaker training. The code below will create, or if it exists, use, the default bucket.
- The IAM role to give Autopilot access to your data. See the Amazon SageMaker documentation for more information on IAM roles: https://docs.aws.amazon.com/sagemaker/latest/dg/security-iam.html

In [10]:
import sagemaker
import boto3
import os
from time import gmtime, strftime, sleep
import pandas as pd

boto_session = boto3.Session(aws_access_key_id=os.environ["ACCESS_ID"],
         aws_secret_access_key= os.environ["ACCESS_KEY"])

region = boto_session.region_name

session = sagemaker.Session(boto_session=boto_session)
bucket = session.default_bucket()
prefix = 'sagemaker/autopilot-dm/link-prediction-2020-08-17'

role = "arn:aws:iam::715633473519:role/mark-sagemaker"

sm = boto_session.client(service_name='sagemaker',region_name=region)

## Downloading the dataset<a name="Downloading"></a>
Download the [direct marketing dataset](!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip) from the sample data s3 bucket. 

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [6]:
# !wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
# !unzip -o bank-additional.zip

local_data_path = 'bank-additional-full.csv'


## Upload the dataset to Amazon S3<a name="Uploading"></a>

Before you run Autopilot on the dataset, first perform a check of the dataset to make sure that it has no obvious errors. The Autopilot process can take long time, and it's generally a good practice to inspect the dataset before you start a job. This particular dataset is small, so you can inspect it in the notebook instance itself. If you have a larger dataset that will not fit in a notebook instance memory, inspect the dataset offline using a big data analytics tool like Apache Spark. [Deequ](https://github.com/awslabs/deequ) is a library built on top of Apache Spark that can be helpful for performing checks on large datasets. Autopilot is capable of handling datasets up to 5 GB.


Read the data into a Pandas data frame and take a look.

In [5]:
training_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

display(training_df)
display(test_df)

,node1,node2,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
0,36751,258427,1,3.0,20.0,6.0,5,7,0.700000,0.833333,1.0,1.0
1,171003,205575,0,1.0,14.0,8.0,1,10,0.476190,1.000000,1.0,1.0
2,78481,90646,1,5.0,36.0,7.0,15,15,1.000000,1.000000,1.0,1.0
3,43367,43368,1,2.0,18.0,7.0,3,4,0.266667,1.000000,1.0,1.0
4,10954,209715,0,2.0,9.0,4.0,0,3,0.000000,1.000000,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
162187,214300,248884,0,1.0,8.0,5.0,1,4,0.666667,1.000000,1.0,0.0
162188,66073,257891,0,0.0,32.0,12.0,6,20,0.714286,1.000000,1.0,0.0
162189,140350,140007,0,0.0,33.0,14.0,1,7,0.127273,0.333333,0.0,0.0
162190,1051,36834,0,0.0,80.0,24.0,6,22,0.115789,1.000000,0.0,0.0


,node1,node2,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
0,27291,196183,0,0.0,88.0,46.0,0,284,0.304440,1.000000,0.0,0.0
1,90831,145595,0,1.0,20.0,8.0,4,10,0.833333,1.000000,0.0,0.0
2,161538,245175,1,4.0,50.0,11.0,10,11,0.244444,1.000000,1.0,1.0
3,196260,196262,1,2.0,9.0,4.0,3,3,1.000000,1.000000,1.0,1.0
4,156543,238771,0,1.0,24.0,9.0,6,6,0.333333,1.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
148251,123126,151401,0,0.0,30.0,17.0,1,42,0.323810,1.000000,0.0,0.0
148252,29089,175602,0,0.0,9.0,10.0,0,2,0.000000,0.055556,0.0,0.0
148253,179621,145287,0,0.0,108.0,39.0,3,41,0.068254,1.000000,0.0,0.0
148254,101812,227196,1,4.0,70.0,15.0,10,23,0.252747,1.000000,1.0,1.0


Note that there are 20 features to help predict the target column 'y'.

Amazon SageMaker Autopilot takes care of preprocessing your data for you. You do not need to perform conventional data preprocssing techniques such as handling missing values, converting categorical features to numeric features, scaling data, and handling more complicated data types.

Moreover, splitting the dataset into training and validation splits is not necessary. Autopilot takes care of this for you. You may, however, want to split out a test set. That's next, although you use it for batch inference at the end instead of testing the model.


### Reserve some data for calling batch inference on the model

Divide the data into training and testing splits. The training split is used by SageMaker Autopilot. The testing split is reserved to perform inference using the suggested model.


In [39]:
# train_data = data.sample(frac=0.8,random_state=200)

# test_data = data.drop(train_data.index)

# test_data_no_target = test_data.drop(columns=['y'])

train_columns = [
    "cn", "pa", "tn", "minTriangles", "maxTriangles", "minCoefficient", "maxCoefficient", "sp", "sl", "label"
]

test_columns = [
    "cn", "pa", "tn", "minTriangles", "maxTriangles", "minCoefficient", "maxCoefficient", "sp", "sl"
]

display(training_df)
display(training_df.drop(columns=["node1", "node2"]))
train_data = training_df.drop(columns=["node1", "node2"])[train_columns]
display(train_data)


display(test_df)
test_data_no_target = test_df.drop(columns=["node1", "node2", "label"])[test_columns]

display(test_data_no_target)

,node1,node2,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
0,36751,258427,1,3.0,20.0,6.0,5,7,0.700000,0.833333,1.0,1.0
1,171003,205575,0,1.0,14.0,8.0,1,10,0.476190,1.000000,1.0,1.0
2,78481,90646,1,5.0,36.0,7.0,15,15,1.000000,1.000000,1.0,1.0
3,43367,43368,1,2.0,18.0,7.0,3,4,0.266667,1.000000,1.0,1.0
4,10954,209715,0,2.0,9.0,4.0,0,3,0.000000,1.000000,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
162187,214300,248884,0,1.0,8.0,5.0,1,4,0.666667,1.000000,1.0,0.0
162188,66073,257891,0,0.0,32.0,12.0,6,20,0.714286,1.000000,1.0,0.0
162189,140350,140007,0,0.0,33.0,14.0,1,7,0.127273,0.333333,0.0,0.0
162190,1051,36834,0,0.0,80.0,24.0,6,22,0.115789,1.000000,0.0,0.0


,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
0,1,3.0,20.0,6.0,5,7,0.700000,0.833333,1.0,1.0
1,0,1.0,14.0,8.0,1,10,0.476190,1.000000,1.0,1.0
2,1,5.0,36.0,7.0,15,15,1.000000,1.000000,1.0,1.0
3,1,2.0,18.0,7.0,3,4,0.266667,1.000000,1.0,1.0
4,0,2.0,9.0,4.0,0,3,0.000000,1.000000,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
162187,0,1.0,8.0,5.0,1,4,0.666667,1.000000,1.0,0.0
162188,0,0.0,32.0,12.0,6,20,0.714286,1.000000,1.0,0.0
162189,0,0.0,33.0,14.0,1,7,0.127273,0.333333,0.0,0.0
162190,0,0.0,80.0,24.0,6,22,0.115789,1.000000,0.0,0.0


,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl,label
0,3.0,20.0,6.0,5,7,0.700000,0.833333,1.0,1.0,1
1,1.0,14.0,8.0,1,10,0.476190,1.000000,1.0,1.0,0
2,5.0,36.0,7.0,15,15,1.000000,1.000000,1.0,1.0,1
3,2.0,18.0,7.0,3,4,0.266667,1.000000,1.0,1.0,1
4,2.0,9.0,4.0,0,3,0.000000,1.000000,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...
162187,1.0,8.0,5.0,1,4,0.666667,1.000000,1.0,0.0,0
162188,0.0,32.0,12.0,6,20,0.714286,1.000000,1.0,0.0,0
162189,0.0,33.0,14.0,1,7,0.127273,0.333333,0.0,0.0,0
162190,0.0,80.0,24.0,6,22,0.115789,1.000000,0.0,0.0,0


,node1,node2,label,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
0,27291,196183,0,0.0,88.0,46.0,0,284,0.304440,1.000000,0.0,0.0
1,90831,145595,0,1.0,20.0,8.0,4,10,0.833333,1.000000,0.0,0.0
2,161538,245175,1,4.0,50.0,11.0,10,11,0.244444,1.000000,1.0,1.0
3,196260,196262,1,2.0,9.0,4.0,3,3,1.000000,1.000000,1.0,1.0
4,156543,238771,0,1.0,24.0,9.0,6,6,0.333333,1.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
148251,123126,151401,0,0.0,30.0,17.0,1,42,0.323810,1.000000,0.0,0.0
148252,29089,175602,0,0.0,9.0,10.0,0,2,0.000000,0.055556,0.0,0.0
148253,179621,145287,0,0.0,108.0,39.0,3,41,0.068254,1.000000,0.0,0.0
148254,101812,227196,1,4.0,70.0,15.0,10,23,0.252747,1.000000,1.0,1.0


,cn,pa,tn,minTriangles,maxTriangles,minCoefficient,maxCoefficient,sp,sl
0,0.0,88.0,46.0,0,284,0.304440,1.000000,0.0,0.0
1,1.0,20.0,8.0,4,10,0.833333,1.000000,0.0,0.0
2,4.0,50.0,11.0,10,11,0.244444,1.000000,1.0,1.0
3,2.0,9.0,4.0,3,3,1.000000,1.000000,1.0,1.0
4,1.0,24.0,9.0,6,6,0.333333,1.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...
148251,0.0,30.0,17.0,1,42,0.323810,1.000000,0.0,0.0
148252,0.0,9.0,10.0,0,2,0.000000,0.055556,0.0,0.0
148253,0.0,108.0,39.0,3,41,0.068254,1.000000,0.0,0.0
148254,4.0,70.0,15.0,10,23,0.252747,1.000000,1.0,1.0


### Upload the dataset to Amazon S3
Copy the file to Amazon Simple Storage Service (Amazon S3) in a .csv format for Amazon SageMaker training to use.

In [40]:
train_file = 'train_data_link-pred-ordered.csv';
train_data.to_csv(train_file, index=False, header=True)
train_data_s3_path = session.upload_data(path=train_file, key_prefix=prefix + "/train")
print('Train data uploaded to: ' + train_data_s3_path)

test_file = 'test_data_link-pred-ordered.csv';
test_data_no_target.to_csv(test_file, index=False, header=False)
test_data_s3_path = session.upload_data(path=test_file, key_prefix=prefix + "/test")
print('Test data uploaded to: ' + test_data_s3_path)

Train data uploaded to: s3://sagemaker-us-east-1-715633473519/sagemaker/autopilot-dm/link-prediction-2020-08-17/train/train_data_link-pred-ordered.csv
Test data uploaded to: s3://sagemaker-us-east-1-715633473519/sagemaker/autopilot-dm/link-prediction-2020-08-17/test/test_data_link-pred-ordered.csv


## Setting up the SageMaker Autopilot Job<a name="Settingup"></a>

After uploading the dataset to Amazon S3, you can invoke Autopilot to find the best ML pipeline to train a model on this dataset. 

The required inputs for invoking a Autopilot job are:
* Amazon S3 location for input dataset and for all output artifacts
* Name of the column of the dataset you want to predict (`y` in this case) 
* An IAM role

Currently Autopilot supports only tabular datasets in CSV format. Either all files should have a header row, or the first file of the dataset, when sorted in alphabetical/lexical order, is expected to have a header row.

In [41]:
input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/train'.format(bucket,prefix)
        }
      },
      'TargetAttributeName': 'label'
    }
  ]

automl_job_config = {
    "CompletionCriteria": {
        "MaxRuntimePerTrainingJobInSeconds": 300,
        "MaxCandidates": 2,
    }
}

output_data_config = {
    'S3OutputPath': 's3://{}/{}/output'.format(bucket,prefix)
  }

You can also specify the type of problem you want to solve with your dataset (`Regression, MulticlassClassification, BinaryClassification`). In case you are not sure, SageMaker Autopilot will infer the problem type based on statistics of the target column (the column you want to predict). 

You have the option to limit the running time of a SageMaker Autopilot job by providing either the maximum number of pipeline evaluations or candidates (one pipeline evaluation is called a `Candidate` because it generates a candidate model) or providing the total time allocated for the overall Autopilot job. Under default settings, this job takes about four hours to run. This varies between runs because of the nature of the exploratory process Autopilot uses to find optimal training parameters.

## Launching the SageMaker Autopilot Job<a name="Launching"></a>

You can now launch the Autopilot job by calling the `create_auto_ml_job` API. 

In [42]:
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-link-pred-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=automl_job_config,
                      RoleArn=role)

AutoMLJobName: automl-link-pred-17-15-15-36


{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:715633473519:automl-job/automl-link-pred-17-15-15-36',
 'ResponseMetadata': {'RequestId': '41ec83fc-be7c-475c-ab92-fe39603eef1b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '41ec83fc-be7c-475c-ab92-fe39603eef1b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Mon, 17 Aug 2020 15:15:36 GMT'},
  'RetryAttempts': 0}}

## Tracking SageMaker Autopilot job progress<a name="Tracking"></a>
SageMaker Autopilot job consists of the following high-level steps : 
* Analyzing Data, where the dataset is analyzed and Autopilot comes up with a list of ML pipelines that should be tried out on the dataset. The dataset is also split into train and validation sets.
* Feature Engineering, where Autopilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* Model Tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [45]:
print ('JobStatus - Secondary Status')
print('------------------------------')


describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
job_run_status = describe_response['AutoMLJobStatus']
    
while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response['AutoMLJobStatus']
    
    print (describe_response['AutoMLJobStatus'] + " - " + describe_response['AutoMLJobSecondaryStatus'])
    sleep(30)

JobStatus - Secondary Status
------------------------------
Completed - MaxCandidatesReached


## Results

Now use the describe_auto_ml_job API to look up the best candidate selected by the SageMaker Autopilot job. 

In [46]:
# auto_ml_job_name = "automl-link-pred-17-13-16-16"

best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_name = best_candidate['CandidateName']
print(best_candidate)
print('\n')
print("CandidateName: " + best_candidate_name)
print("FinalAutoMLJobObjectiveMetricName: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("FinalAutoMLJobObjectiveMetricValue: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

{'CandidateName': 'tuning-job-1-ef559623ff8246c4a8-002-0b671f2e', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy', 'Value': 0.9499199986457825}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:715633473519:processing-job/db-1-ab7a013ae9d642afbea8f8d1b3d2bb76ff2dd3f4ee77443596a2d50643', 'CandidateStepName': 'db-1-ab7a013ae9d642afbea8f8d1b3d2bb76ff2dd3f4ee77443596a2d50643'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:715633473519:training-job/automl-lin-dpp1-1-af26ffc85fc64298a8dd1d785ba5f3ff8ec72da741824', 'CandidateStepName': 'automl-lin-dpp1-1-af26ffc85fc64298a8dd1d785ba5f3ff8ec72da741824'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:715633473519:transform-job/automl-lin-dpp1-csv-1-4a7b321119cd427ebd2e2445288372b5f5cf9728c', 'CandidateStepNa

### Perform batch inference using the best candidate

Now that you have successfully completed the SageMaker Autopilot job on the dataset, create a model from any of the candidates by using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). 

In [47]:
# timestamp_suffix = "17-10-44-29"
model_name = 'automl-link-pred-model-' + timestamp_suffix

model = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Model ARN corresponding to the best candidate is : {}'.format(model['ModelArn']))

Model ARN corresponding to the best candidate is : arn:aws:sagemaker:us-east-1:715633473519:model/automl-link-pred-model-17-15-15-36


You can use batch inference by using Amazon SageMaker batch transform. The same model can also be deployed to perform online inference using Amazon SageMaker hosting.

In [48]:
transform_job_name = 'automl-link-pred-transform-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': test_data_s3_path
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results'.format(bucket,prefix),
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources
)

{'TransformJobArn': 'arn:aws:sagemaker:us-east-1:715633473519:transform-job/automl-link-pred-transform-17-15-15-36',
 'ResponseMetadata': {'RequestId': '43969c4c-0876-4e4d-a60b-aaf9ab53d7f5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '43969c4c-0876-4e4d-a60b-aaf9ab53d7f5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '115',
   'date': 'Mon, 17 Aug 2020 16:34:26 GMT'},
  'RetryAttempts': 0}}

Watch the transform job for completion.

In [49]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


Now let's view the results of the transform job:

In [50]:
s3_output_key = '{}/inference-results/test_data_link-pred-ordered.csv.out'.format(prefix);
print(s3_output_key)
local_inference_results_path = 'inference_results_link-pred-ordered.csv'

inference_results_bucket = boto_session.resource("s3").Bucket(session.default_bucket())

inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=';', header=None)
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

sagemaker/autopilot-dm/link-prediction-2020-08-17/inference-results/test_data_link-pred-ordered.csv.out


,0
0,0
1,0
2,1
3,1
4,0
...,...
148251,0
148252,0
148253,0
148254,1


### View other candidates explored by SageMaker Autopilot
You can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by SageMaker Autopilot and sort them by their final performance metric.

In [54]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, SortBy='FinalObjectiveMetricValue')['Candidates']
index = 1
for candidate in candidates:
  print (str(index) + "  " + candidate['CandidateName'] + "  " + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))
  index += 1

1  tuning-job-1-ef559623ff8246c4a8-002-0b671f2e  0.9499199986457825
2  tuning-job-1-ef559623ff8246c4a8-001-f3dfff59  0.948930025100708


### Candidate Generation Notebook
    
Sagemaker AutoPilot also auto-generates a Candidate Definitions notebook. This notebook can be used to interactively step through the various steps taken by the Sagemaker Autopilot to arrive at the best candidate. This notebook can also be used to override various runtime parameters like parallelism, hardware used, algorithms explored, feature extraction scripts and more.
    
The notebook can be downloaded from the following Amazon S3 location:

In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['CandidateDefinitionNotebookLocation']


### Data Exploration Notebook
Sagemaker Autopilot also auto-generates a Data Exploration notebook, which can be downloaded from the following Amazon S3 location:

In [ ]:
sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['AutoMLJobArtifacts']['DataExplorationNotebookLocation']


## Cleanup

The Autopilot job creates many underlying artifacts such as dataset splits, preprocessing scripts, or preprocessed data, etc. This code, when un-commented, deletes them. This operation deletes all the generated models and the auto-generated notebooks as well. 

In [ ]:
#s3 = boto3.resource('s3')
#bucket = s3.Bucket(bucket)

#job_outputs_prefix = '{}/output/{}'.format(prefix,auto_ml_job_name)
#bucket.objects.filter(Prefix=job_outputs_prefix).delete()

In [23]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score


def evaluate_model(predictions, actual):
    return pd.DataFrame({
        "Measure": ["Accuracy", "Precision", "Recall"],
        "Score": [accuracy_score(actual, predictions), 
                  precision_score(actual, predictions), 
                  recall_score(actual, predictions)]
    })

def feature_importance(columns, classifier):        
    display("Feature Importance")
    df = pd.DataFrame({
        "Feature": columns,
        "Importance": classifier.feature_importances_
    })
    df = df.sort_values("Importance", ascending=False)    
    ax = df.plot(kind='bar', x='Feature', y='Importance', legend=None)
    ax.xaxis.set_label_text("")
    plt.tight_layout()
    plt.show()

In [53]:
predictions = data[0]
y_test = test_df["label"]


display(evaluate_model(y_test, predictions))
# display(precision_score(y_test, predictions, average="binary", pos_label="yes"))
# display(recall_score(y_test, predictions, average="binary", pos_label="yes"))
# display(evaluate_model(y_test, predictions))

,Measure,Score
0,Accuracy,0.962477
1,Precision,0.961364
2,Recall,0.963509
